In [57]:
import nltk
import numpy as np

In [58]:
from nltk.stem import WordNetLemmatizer
from sklearn.linear_model import LogisticRegression
from bs4 import BeautifulSoup

In [8]:
wordnet_lemmatizer = WordNetLemmatizer()

In [9]:
stopwords = set(w.rstrip() for w in open('stopwords.txt'))

In [11]:
# Load positive reviews
positive_reviews = BeautifulSoup(open('electronics/positive.review').read())
positive_reviews = positive_reviews.findAll('review_text')

C:\Anaconda3\envs\tensorflow\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Anaconda3\envs\tensorflow\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


In [14]:
# Load negative reviews
negative_reviews = BeautifulSoup(open('electronics/negative.review').read())
negative_reviews = negative_reviews.findAll('review_text')

C:\Anaconda3\envs\tensorflow\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Anaconda3\envs\tensorflow\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


In [26]:
# Preprocess string before training
# Lower character
# Remove any character shorter than 2 
# Lemmatizer all the character: dogs -> dog, cats -> cat, goes -> go
def my_tokenizer(s):
    s = s.lower()
    tokens = nltk.tokenize.word_tokenize(s)
    tokens = [t for t in tokens if len(t) > 2]
    tokens = [wordnet_lemmatizer.lemmatize(t) for t in tokens]
    tokens = [t for t in tokens if t not in stopwords]
    return tokens

In [29]:
# Create a dictionary of all the words in the data
word_index_map = {}
current_index = 0

In [30]:
# Store all the token of all the comment
positive_tokenized = []
negative_tokenized = []

In [31]:
# Store all the words appear in the data into a list 
# and tokenized all the reviews in positive data  
for review in positive_reviews:
    tokens = my_tokenizer(review.text)
    positive_tokenized.append(tokens)
    for token in tokens:
        if token not in word_index_map:
            word_index_map[token] = current_index
            current_index += 1

In [36]:
# Store all the words appear in the data into a list 
# and tokenized all the reviews in negative data 
for review in negative_reviews:
    tokens = my_tokenizer(review.text)
    negative_tokenized.append(tokens)
    for token in tokens:
        if token not in word_index_map:
            word_index_map[token] = current_index
            current_index += 1

In [67]:
# Apply word propotion method words-frequency
# Calculate the propotion words appear in data
# devide by the size of data 
def tokens_to_vector(tokens, label):
    x = np.zeros(len(word_index_map) + 1)
    for t in tokens:
        i = word_index_map[t]
        x[i] += 1
    x = x / x.sum()
    x[-1] = label
    return x

In [47]:
# N is the size of numpy array which contents all the vector of each review
N = len(positive_tokenized) + len(negative_tokenized)
data = np.zeros((N, len(word_index_map) + 1))

In [49]:
i = 0 # Set counter to 0
# Convent each group of token from each reviews
for tokens in positive_tokenized:
    xy = tokens_to_vector(tokens, 1)
    data[i,:] = xy
    i += 1

In [50]:
# Convert each group of token from each reviews
for tokens in negative_tokenized:
    xy = tokens_to_vector(tokens, 0)
    data[i,:] = xy
    i += 1

In [51]:
# Shuffle data for training 
np.random.shuffle(data)

In [77]:
print("Original size", data.shape)
print("Training size", data[:, :-1].shape)

Original size (3000, 11093)
Training size (3000, 11092)


In [78]:
X = data[:, :-1]    # all the row and the first 11093 column
Y = data[:, -1]     # all the row The last column

In [53]:
Xtrain = X[:-100,]
Ytrain = Y[:-100,]
Xtest = X[-100:,]
Ytest = Y[-100:,]

## Machine Learning Model

#### Logistic Regression

In [84]:
# Logistic Regression
model = LogisticRegression()
model.fit(Xtrain, Ytrain)
print("Logistic Regression classification rate:", model.score(Xtest, Ytest))

Logistic Regression classification rate: 0.7


In [85]:
# Threshold value decide the Sentiment rate 
threshold = 0.5
for word, index in word_index_map.items():
    weight = model.coef_[0][index]
    if weight > threshold or weight < -threshold:
        print(word, weight)

product 0.52730448667
you 1.11786435437
perfect 1.37322797854
service -0.507426666792
minute -0.522122327578
fit 0.610990770393
speaker 1.16419928874
paper 0.85816521095
static -0.679916546735
junk -0.700629618267
cable 0.869119544871
highly 1.30449649962
look 0.710153477784
stopped -0.689903334996
return -1.51576104822
card -0.63700967985
value 0.740541664751
awesome 0.532390520531
support -1.14136809227
wa -1.90539694547
excellent 1.8613991542
error -0.598855795424
software -0.571498544506
memory 1.29871543102
tried -1.03102564181
expected 0.753830872391
bad -1.03367212383
refund -0.818151700414
disappointed -0.605718218361
ha 1.05730693022
money -1.33620437482
little 1.27265201325
fast 1.14988458051
home 0.708400995892
pleased 0.624484414966
piece -0.595716488412
feature 0.599780210298
comfortable 0.845951928933
returned -0.88995507328
unfortunately -0.516349487511
waste -1.27769189174
overall 0.608155488381
item -1.23073905034
photo 0.509641340915
lot 0.865689573299
month -0.816731

#### Naive Bayes

In [93]:
from sklearn.naive_bayes import MultinomialNB

In [94]:
model = MultinomialNB()
model.fit(Xtrain, Ytrain)
print("Naive Bayes classification rate:", model.score(Xtest, Ytest))

Naive Bayes classification rate: 0.7


In [95]:
# Threshold value decide the Sentiment rate 
threshold = 0.5
for word, index in word_index_map.items():
    weight = model.coef_[0][index]
    if weight > threshold or weight < -threshold:
        print(word, weight)

deliterious -9.39781529401
802.11b -9.36553763509
m130 -9.39781529401
wise -9.36358187602
obviously -9.29634506334
zenith -9.31777258634
sturdy -9.09396583734
sansa -9.33527989469
maxtor -9.39781529401
awaits -9.37719600681
chyrsler -9.37979678851
online/offline -9.35699329949
product -7.01377358884
consume -9.39781529401
hardcore -9.38439227368
recived -9.39781529401
mccartney -9.38368791475
scammed -9.39781529401
journalist -9.36882775714
john -9.38203891556
appointment -9.36400317723
affair -9.39781529401
worker -9.39781529401
unknown -9.39781529401
millenium -9.39781529401
herbert -9.39385489279
iron-on -9.33688905856
sealing -9.37801266671
sonys -9.39781529401
sychronized -9.39781529401
mbps -9.39781529401
chinese -9.39374196862
reduce -9.37839720815
refer -9.38796299757
throwaway -9.39026808838
date/time -9.38706250223
thirdly -9.39781529401
pictue -9.28003225835
1973 -9.39578483646
kitchen -9.18447091918
traveller -9.36144764984
throughput -9.36144764984
crank -9.26183721789
tra

psc -9.39781529401
cd-player -9.38944704434
tremendo -9.32370732186
enables -9.36144764984
catch -9.29298968075
proticol -9.39781529401
finally -8.95707399109
dug -9.39781529401
entirely -9.34675598421
drill -9.39306469125
grundig -9.39781529401
expedient -9.39781529401
microphone -9.26069187559
38th -9.39781529401
phillips -9.37534243816
emailed -9.37174464622
auto-eject -9.39781529401
iterm -9.37312268142
drawback -9.23892145341
non-grounded -9.379123161
tricycle -9.39781529401
suppressed -9.39781529401
cobham -9.39578483646
jbl -9.12513973995
floorboard -9.37839720815
pissed -9.39781529401
sear -9.379123161
kudos -9.35968280752
//www.radarbusters.com/ -9.39781529401
nikon -9.36391374234
plated -9.38254782188
4-5 -9.39781529401
equipment -9.0504095572
sixty -9.39781529401
magellen -9.39781529401
closer -9.38937642537
outgoing -9.37416997583
nics -9.37183980761
affluent -9.3940769719
manning -9.39781529401
dance -9.38420964196
v=glance -9.39781529401
aesthetic -9.38958730305
advertize

helpline -9.39781529401
intelligible -9.39781529401
120-watt -9.39781529401
receiption -9.32882242252
oft -9.39781529401
obtains -9.37719600681
-you -9.39781529401
fiber -9.35627837839
affixed -9.31080391702
autofire -9.39781529401
investigation -9.38612893454
1khz -9.37946615534
videotape -9.39781529401
hyphen -9.38598083636
tug -9.3896809778
cancellation -9.38072086065
slight -9.24407489834
boomy -9.32304596819
disturbed -9.39781529401
shuts -9.39781529401
astonished -9.39781529401
plasma -9.17707614824
perspective -9.38598083636
510 -9.36144764984
message -9.00109741439
consummes -9.39781529401
anothe -9.39781529401
acrobat -9.38712000489
acurate -9.35699329949
stopped -9.27878877231
field -9.24094278648
reflection -9.39781529401
comparing -9.37041631982
mary -9.39781529401
256 -9.26339646194
dimensiones -9.37979678851
straw -9.39781529401
loc=3 -9.39259335003
sdsdh-1024-901 -9.34374807274
recreation -9.39471451633
colored -9.34663903244
powerex -9.35129527838
grammar -9.39781529401

emap -9.36144764984
pricetag -9.39781529401
40. -9.38300020823
rudeness -9.39781529401
din -9.38688622348
snug -9.31309071148
dragged -9.37312268142
supporting -9.39781529401
alter -9.39285250467
rendering -9.38185935369
foremost -9.39112630586
verbatim -9.24777887168
thye -9.36704363534
w2000 -9.39781529401
champion -9.37839720815
legacy -9.39781529401
tau -9.37676188481
alwayz -9.39781529401
pioneer -9.37740479515
fire -9.30837824153
factory -9.31679429335
albeit -9.38651573876
detroit -9.39781529401
gamepad -9.32790162568
mazda -9.39781529401
penatration -9.38796299757
practicalness -9.39781529401
supremecy -9.3881533831
htz -9.33327677287
shelter -9.39781529401
metiocracy -9.39781529401
wa -6.64585488107
wlancfg5.exe -9.38155477314
con -8.90859533095
viewsonic -9.25900006189
2.4gig -9.38457006726
blinked -9.39242444538
daylight -9.38507626823
positioned -9.38625324248
pun -9.3881533831
instrument -9.37655623426
aup -9.36882775714
'far -9.39781529401
nwb -9.39781529401
720p -9.39282

two+ -9.39781529401
blessfully -9.39205158929
anti -9.38625447161
scold -9.36964441704
cite -9.39074812679
sixth -9.39781529401
unwooded -9.39781529401
identifying -9.38786496316
30g -9.379123161
cd/dvd -9.35457239562
imagine -9.22797320502
directv -9.34439850606
deducted -9.39781529401
audio-enhanced -9.37876709904
latter -9.39781529401
lover -9.38805911907
surely -9.32324915969
reducing -9.39781529401
hope -9.24922505704
proven -9.33439346757
fail -9.34303439753
double-seam -9.39084662469
+fits -9.37041631982
eq50 -9.38805911907
establish -9.37979678851
purport -9.39781529401
hybrid -9.39084662469
demonstration -9.38965198337
digitial -9.31777258634
re-organize -9.39781529401
pant -9.37311985815
brick-and-mortar -9.3928277525
3600 -9.39781529401
perform -9.18900842047
e-bay -9.39781529401
conversion -9.34141474303
maine -9.38745138823
moderate -9.35668165974
laced -9.39781529401
silicon -9.39781529401
throughly -9.39781529401
spaced -9.37963793975
minidiscs -9.38819983531
aaron -9.39

...

ring -9.08037209
steel -9.36972910898
over-the-air -9.29245477835
disappear -9.36391374234
wan -9.39205158929
inconsistently -9.39781529401
skipdoctor -9.38766292255
6800ultra -9.38868281045
re-recording -9.39781529401
cfii -9.39781529401
attracts -9.38824584299
price -7.03901777844
9/05 -9.35336353144
insensitive -9.39781529401
game -8.62066213717
animal -9.38723652637
verto -9.39218147629
wow -9.04402444689
mag -9.37979678851
winamp -9.37979537677
studio -9.30862690946
encoding -9.39158474426
select -9.29438289499
united -9.3765148594
massive -9.35180069051
1800mah -9.39781529401
strength -9.36025265949
ghz -9.330076503
wish -8.73132336381
blowing -9.39781529401
superadio -9.39781529401
lennon -9.38770398441
fate -9.39781529401
tightly -9.34050657882
sr-60 -9.36502547119
eyeing -9.39433702963
outlined -9.39781529401
mutter -9.39385489279
1-disable -9.39112630586
virtual -9.39084662469
d-link -9.16532628943
understandable -9.38200979294
lookin -9.38507626823
markt -9.39781529401
shipp

offset -9.35765636719
repositionable -9.39781529401
mail -9.33556476193
128 -9.19947079352
re-establish -9.39781529401
aisle -9.39188055849
splashed -9.39781529401
1100 -9.39781529401
imitation -9.37041631982
job -8.69502299441
candle -9.39084662469
v6s -9.38651573876
competitor -9.32781809611
vibrant -9.3501038786
3.99 -9.39781529401
somone -9.38984712436
meaning -9.37271469108
nokia -9.08169528624
costly -9.39781529401
6.1 -9.36144764984
sandisk.com -9.39374196862
straighten -9.39433702963
encrusted -9.39781529401
contact -9.16741385122
elderly -9.35336353144
backspace -9.3660665957
modem -9.03123573194
favorite -9.17089080621
200mhz -9.39306469125
forensic -9.39781529401
glossy -9.0990583385
n-track -9.38598083636
600+ -9.39218147629
bsod -9.39781529401
lanning -9.36007496603
sb75g2 -9.39218147629
-signal -9.39781529401
watergate -9.3920846193
solitaire -9.38382905204
multiple -9.05619192796
unit.. -9.39781529401
successfully -9.3528805563
volt -9.32312932742
10-digit -9.39781529401

fabric -9.39781529401
unprepared -9.38254782188
ruling -9.39781529401
availible -9.36294045145
shaped -9.36027395706
rei -9.39781529401
injury -9.39781529401
allows -9.02731680302
blame -9.37615644183
vigorous -9.39370851206
sparatic -9.39781529401
unit -7.93802850909
mean -9.08607042321
500 -9.1913340921
megapixels -9.39117075129
asa -9.37979678851
proximity -9.38382905204
mdr-nc20 -9.39433702963
astonishing -9.36796233086
mesh -9.3940769719
jogging -9.29219649721
cranium -9.38997211655
reinforcement -9.38128599206
satisfaction -9.31756143769
it.. -9.39781529401
bandwagon -9.37839720815
precisely -9.33488747671
base -8.6937286038
upwards -9.37876709904
forcing -9.39781529401
59.99 -9.39781529401
belonging -9.3804235513
ant -9.29067065656
blaze -9.39158474426
substantially -9.39781529401
25-15,600 -9.3881533831
st. -9.39781529401
beanbag -9.39781529401
undisclosed -9.39781529401
e30s -9.35129527838
scuffing -9.36964441704
dismount -9.39781529401
complemented -9.39394681723
sputter -9.3

cuenta -9.37979678851
de-install -9.38965198337
mileage -9.38598083636
mousepad -9.3804235513
1.12 -9.39781529401
tivo -9.23263147876
crumbled -9.39781529401
finder -9.39781529401
model-specific -9.39781529401
orgaized -9.39781529401
toothpick -9.39781529401
hardly -9.35500487057
adapts -9.38474321244
45min -9.39781529401
configuring -9.3742847966
shooting -9.35426842318
intend -9.37915242015
bitrate -9.39433702963
metro -9.39781529401
8/26 -9.39781529401
irqs -9.39781529401
disrupt -9.39781529401
haha -9.39781529401
recieved -9.28194494886
backordered -9.39781529401
accessory -9.08673020197
casette -9.39781529401
14-20 -9.38128599206
uncolored -9.37041631982
initially -9.27913711714
overriding -9.39781529401
squalling -9.35699329949
megapixel -9.3255651853
construction -9.16758629814
rzr -9.36391374234
2007. -9.38507626823
recent -9.34181801732
all-channel -9.39781529401
ran -9.18024970522
steal-me -9.39781529401
dvpnss550/b -9.37114704693
attic -9.39306469125
ramp -9.38474321244
soft

299 -9.3852365118
electical -9.39781529401
professionally -9.39781529401
lifting -9.39781529401
payn -9.39781529401
discontinue -9.39781529401
beautifully -9.22894671569
blink -9.39781529401
virtually -9.36796233086
1/8 -9.37970511528
dump -9.39781529401
trackman -9.33166401381
melody -9.35129527838
figure -9.14562625148
expert -9.34043119025
flexibility -9.39781529401
indoors -9.39471451633
cd-r -9.21366815042
mp3-player -9.37041631982
sennhesier -9.39578483646
house -8.85499374792
released -9.37719600681
2-500 -9.39781529401
roaring -9.37801266671
suppose -9.30677430013
accumulated -9.39242444538
initiallize -9.39781529401
airtunes -9.31080391702
speakers/headphones -9.39321758476
specfic -9.39781529401
damaging -9.34919164387
vaseline -9.38491188918
steely -9.39578483646
care -8.9780232904
infuriated -9.39781529401
online -9.1179466162
wor -9.39781529401
seek -9.39121460998
virgin -9.38745138823
iaudio -9.39781529401
borrow -9.39781529401
on-ear -9.3881533831
isao -9.39578483646
swi

#### Support Vector Machine

In [96]:
from sklearn import svm

In [101]:
model = svm.LinearSVC()
model.fit(Xtrain, Ytrain)
print("SVN classification rate:", model.score(Xtest, Ytest))

SVN classification rate: 0.78


In [104]:
# Threshold value decide the Sentiment rate 
threshold = 0.5
for word, index in word_index_map.items():
    weight = model.coef_[0][index]
    if weight > threshold or weight < -threshold:
        print(word, weight)

obviously -0.575194885651
toner 0.622651911916
traveling 0.931124822602
you 0.882878452276
flimsy -0.969575877262
perfect 3.24813751871
suck -1.21464821397
poorly -0.779070834583
combination -0.631448815086
world 0.524120332179
headset 0.655435744506
service -0.820809341986
nicely 0.665940335255
capacity 0.711705613714
stop -1.04849054457
faster 0.805149278938
plenty 0.864588991034
range 0.748485214556
minute -1.01931411179
plug 0.668488023736
finally -0.55019762714
microphone -0.73980100202
nikon -0.637348106783
fantastic 0.632603554353
fit 1.12394565407
speaker 1.44663077886
found 0.534169264538
wear 0.540444850374
loved -0.690943765545
theater 0.719788149014
saying -0.698991695988
awsome 0.775718471734
replacement -0.953702710027
handle -0.615572411959
cool 0.509697356403
reasonably 0.714172178654
planed 0.519714457102
mistake -0.601352464866
paper 1.59901601702
jack -0.780213424554
static -1.64568114067
junk -1.4753126717
cable 1.00278420267
philip -0.676837646703
outstanding 0.810